### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory, DataTypeStr

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

jiangDataDirectory = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps")
jiang2uM_CellularBasename = "BEAS-2B_2uM_BPDE_cell_24h"
jiang2uM_NakedBasename = "BEAS-2B_2uM_BPDE_nDNA_24h"

jiang2uM_CellularDamageMutperiodInputFilePaths = getFilesInDirectory(os.path.join(jiangDataDirectory, jiang2uM_CellularBasename), DataTypeStr.mutations + ".bed")
jiang2uM_NakedDamageMutperiodInputFilePath = getFilesInDirectory(os.path.join(jiangDataDirectory, jiang2uM_NakedBasename), DataTypeStr.mutations + ".bed", searchRecursively = False)

### Run the mutperiod pipeline to count damaged bases relative to transcription start sites.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite, generateCustomBackground

TSS_FilePaths = list()
TSS_FilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_genic_regions_unnamed_TSSs", "hg19_genic_regions_unnamed_TSSs.bed"))

TSS_Names = [os.path.basename(TSS_FilePath).rsplit('.', 1)[0] for TSS_FilePath in TSS_FilePaths]

generateCustomBackground(os.path.dirname(jiang2uM_NakedDamageMutperiodInputFilePath), TSS_Names,
                         useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)

runAnalysisSuite(jiang2uM_CellularDamageMutperiodInputFilePaths, TSS_Names,
                 normalizationMethod = "Custom Background", customBackgroundDir = os.path.dirname(jiang2uM_NakedDamageMutperiodInputFilePath),
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)